<a href="https://colab.research.google.com/github/cristiandarioortegayubro/Ciencia-de-datos-en-la-optimizacion-de-recursos-hidricos/blob/master/Ciencia_de_datos_en_la_optimizacion_de_recursos_hidricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ciencia de datos en la optimización de recursos hídricos**


## **USGS Landsat 8 Surface Reflectance Tier 1**


---



### Librerias y autenticación

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=MvymtE85qIxupQIzFJOLfKtA_xBNtKPELFh36aG7d8Q&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xgFn3IpbQKt5xnfvONfEnDRrJe5974COVeN50vtZFk-BMsn5LSb99xM

Successfully saved authorization token.


*Primer paso, importar Earth Engine (ee) y posteriormente autenticar usuario a traves del link, asociando a una cuenta gmail, generando codigo token el cual debe ser copiado y pegado para lograr autorizacion satisfactoria*

### Mapa interactivo de Mendoza con librería folium

#### Librerias para el mapa interactivo

In [0]:
import sys
import folium
from eeconvert import eeImageToFoliumLayer as ee_plot
import pandas as pd
import webbrowser

#### Localización del mapa y definición de bandas satelitales

In [0]:
potrerillos = [-32.9946, -69.1280]
punto = [-69.1603, -32.9781]

In [0]:
map = folium.Map(location = potrerillos, position='Center', tiles='OpenStreetMap', zoom_start=12.5)
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

#### Conexión con Landsat 8 (Surface Reflectance)

In [0]:
IDcollection = 'LANDSAT/LC08/C01/T1_SR'

In [0]:
l8sr = ee.ImageCollection(IDcollection)
ICFilter = l8sr.filterDate('2018-01-01', '2019-12-28')
GP = ee.Geometry.Point(punto)
ICFilter = ICFilter.filterBounds(GP)

In [62]:
ICFilter.size().getInfo()

44

Este es el tamaño de la colección filtrada.

#### Aplicando función de mascara de nubes

In [0]:
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

#### Definiendo el conjunto de imágenes años 2018 y 2019



In [0]:
image = ICFilter.map(maskL8sr).median()

Reducción del conjunto de imágenes, a una sola imagen

#### Índice de agua de diferencia normalizada (NDWI)


In [0]:
NDWI = image.normalizedDifference(['B3','B6'])

#### Definiendo los filtros de las capas para el mapa

In [66]:
mapid = image.getMapId({'bands': ['B4','B3','B2'], 'min': 0, 'max': 0.3, 'ganma': 1.3})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    show=False,
    name='RGB',
  ).add_to(map),
mapid = image.getMapId({'bands': ['B3'], 'min': 0, 'max': 0.3, 'ganma': 1.3})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    show=False,
    name='Verde',
  ).add_to(map),
mapid = image.getMapId({'bands': ['B6'], 'min': 0, 'max': 0.3})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    show=False,
    name='Infrarrojo',
  ).add_to(map),
mapid = NDWI.getMapId({'min': 0, 'palette':'red, blue'})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    show=False,
    name='NDWI',
  ).add_to(map)

#### Habilitando el control de capas

In [53]:
map.add_child(folium.LayerControl())

#### Creando diferentes herramientas

In [0]:
tooltip1 = 'Clic punto de medición'

#### Habilitando popus para ver latitud y longitud

In [55]:
map.add_child(folium.LatLngPopup())

#### Mostrando el mapa generado

In [0]:
map

In [0]:
import geopandas as gpd